In [ ]:
import torch
import re
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import numpy as np
import matplotlib.pyplot as plt
import random
import io
import os
import torchtext
from tqdm.notebook import tqdm

In [ ]:
with open('gram_minus_ess_new.txt','r') as f:
    e_file = f.read()

with open('gram_minus_ness_new.txt','r') as f1:
    n_file = f1.read()
    
x = re.split('>(DEG\d+)\n([ATGC\n]+)', e_file)
v = [i.replace('\n','') for i in x]

x2 = re.split('>(DNEG\d+)\n([ATGC\n]+)', n_file)
v2 = [i.replace('\n','') for i in x2]

e_names = []
e_seq = []

for i in v:
    match1= re.search('^[ATGC]+$',i)
    match2= re.search('>?DEG',i)
    if match2:
        if not 'available' in i:
            e_names.append(i)
    else:
        if match1:
            e_seq.append(i)

n_names = []
n_seq = []

for i in v2:
    match1= re.search('^[ATGC]+$',i)
    match2= re.search('>?DNEG',i)
    if match2:
        if not 'available' in i:
            n_names.append(i)
    else:
        if match1:
            n_seq.append(i)

In [ ]:
with open('gram_plus_ess_new.txt','r') as f:
    e_file = f.read()

with open('gram_plus_ness_new.txt','r') as f1:
    n_file = f1.read()
    
x = re.split('>(DEG\d+)\n([ATGC\n]+)', e_file)
v = [i.replace('\n','') for i in x]

x2 = re.split('>(DNEG\d+)\n([ATGC\n]+)', n_file)
v2 = [i.replace('\n','') for i in x2]

e_names2 = []
e_seq2 = []

for i in v:
    match1= re.search('^[ATGC]+$',i)
    match2= re.search('>?DEG',i)
    if match2:
        if not 'available' in i:
            e_names2.append(i)
    else:
        if match1:
            e_seq2.append(i)

n_names2 = []
n_seq2 = []

for i in v2:
    match1= re.search('^[ATGC]+$',i)
    match2= re.search('>?DNEG',i)
    if match2:
        if not 'available' in i:
            n_names2.append(i)
    else:
        if match1:
            n_seq2.append(i)

In [ ]:
def seq_with_label(seq, label):
    
    words_labels = []
    
    for s in seq:
            
        words_labels.append([label, s])
            
    return words_labels

In [ ]:
e_data = seq_with_label(e_seq, 'e')
ne_data = seq_with_label(n_seq, 'n')

In [ ]:
e_data2 = seq_with_label(e_seq2, 'e')
ne_data2 = seq_with_label(n_seq2, 'n')

In [ ]:
len(e_data)

13266

In [ ]:
len(e_data2)

5509

In [ ]:
e_len = len(e_data2)

data = e_data[:e_len] + ne_data[:e_len]

In [ ]:
random.shuffle(data2)

In [ ]:
import csv

In [ ]:
with open('genes_with_labels.tsv', 'wt') as file:
    tsv_writer = csv.writer(file, delimiter='\t')
    for elem in data:
      tsv_writer.writerow(elem)

In [ ]:
def tokenizator(seq):
    reprezentacja = []
    for i in range(len(seq)-2):
        reprezentacja.append(seq[i:(i+3)])
    return reprezentacja

In [ ]:
%pip install torchtext==0.9



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user


In [ ]:
import torchtext

In [ ]:
text_field = torchtext.legacy.data.Field(sequential=True,      
                                  tokenize=tokenizator, 
                                  include_lengths=True, 
                                  batch_first=True,
                                  use_vocab=True)      

label_field = torchtext.legacy.data.Field(sequential=False,    
                                   use_vocab=False,     
                                   is_target=True,      
                                   batch_first=True,
                                   preprocessing=lambda x: int(x == 'e')) 

fields = [('label', label_field), ('seq', text_field)]


In [ ]:
train = torchtext.legacy.data.TabularDataset('genes_with_labels.tsv', "tsv", fields)

In [ ]:
text_field.build_vocab(train)

In [ ]:
torchtext_train_dataloader = torchtext.legacy.data.BucketIterator(train,
                                           batch_size=32,
                                           sort_key=lambda x: len(x.seq), 
                                           sort_within_batch=True,        
                                           repeat=False)  

In [ ]:
data2 = e_data2[:e_len] + ne_data2[:e_len]

In [ ]:
with open('genes_with_labels2.tsv', 'wt') as file:
    tsv_writer = csv.writer(file, delimiter='\t')
    for elem in data2:
      tsv_writer.writerow(elem)

In [ ]:
train2 = torchtext.legacy.data.TabularDataset('genes_with_labels2.tsv', "tsv", fields)

In [ ]:
torchtext_train_dataloader2 = torchtext.legacy.data.BucketIterator(train2,
                                           batch_size=32,
                                           sort_key=lambda x: len(x.seq), 
                                           sort_within_batch=True,        
                                           repeat=False)  

In [ ]:
class T_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True) #LSTM
        self.fc = nn.Linear(hidden_size, num_classes)  #przeksztalcenie liniowe
    
    def forward(self, x):
        x = x.unsqueeze(2)
        x = torch.tensor(x)
        x = torch.tensor(x, dtype=torch.float)
        h0 = torch.zeros(1, x.size(0), self.hidden_size) #początkowy  h0
        c0 = torch.zeros(1, x.size(0), self.hidden_size) #początkowy c0
        out, _ = self.lstm(x, (h0, c0))  #LSTM
        out = self.fc(out[:, -1, :]) #przeksztlcam jeszcze liniowo ostatni output
        return out

In [ ]:
lstm_model = T_LSTM(1, 50, 2)

In [ ]:
lstm_model.hidden_size

50

In [ ]:
#Funkcja wyznaczająca dokładność predykcji:

def get_accuracy(model, data_loader):
    data_loader.create_batches()
    correct, total = 0, 0  #ile ok, ile wszystkich

    for num, batch in enumerate(data_loader): #przechodzi dane

        # Put all example.text of batch in single array.
        batch_text = list(batch)[0]
        labels = list(batch)[1]
        x = torch.tensor(batch_text[0])
        output = model(x) #co mowi model
        pred = output.max(1, keepdim=True)[1]  #ktora kategoria
        #print(pred)
        correct += pred.eq(labels.view_as(pred)).sum().item()
        #print(correct)
        total += labels.shape[0]
    return correct / total

In [ ]:
#funkcja do trenowania modelu

def train_model(model, train_loader, valid_loader, num_epochs=5, learning_rate=1e-5):
    criterion = nn.CrossEntropyLoss() #funkcja kosztu
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) #optymalizator modelu
    losses, train_acc, valid_acc, epochs = [], [], [], []  #cztery listy na wartosci funkcji kosztu, dokladnosc na zbiorze testowym i walidacyjnym, numer epoki
    
    for epoch in tqdm(range(num_epochs)): #przechodz kolejne epoki (iteracje)
        
        #train_loader.create_batches()
        
        for num, batch in enumerate(train_loader): #.batches):
            
            # Put all example.text of batch in single array.
            batch_text = list(batch)[0]
            batch_label = list(batch)[1]
            optimizer.zero_grad()
            x = torch.tensor(batch_text[0])
            pred = model(x)
            loss = criterion(pred, batch_label)  #wartosc funkcji kosztu - porownanie tego co mowi model, a tego jak jest
            loss.backward()                  #pochodna po funkcji kosztu
            optimizer.step()                 #aktualizacja parametrow

            
        losses.append(float(loss))           #zapisz aktualną wartosc funkcji kosztu
        epochs.append(epoch)                 #zapisz aktualny numer epoki
        train_acc.append(get_accuracy(model, torchtext_train_dataloader))   #dokladnosc na zbiorze treningowym
        valid_acc.append(get_accuracy(model, torchtext_valid_dataloader))   #dokladnosc na zbiorze walidacyjnym
        print(f'Epoch number: {epoch+1} | Loss value: {loss} | Train accuracy: {round(train_acc[-1],3)} | Valid accuracy: {round(valid_acc[-1],3)}')

In [ ]:
train_model(lstm_model, torchtext_train_dataloader, torchtext_valid_dataloader, num_epochs=5, learning_rate=0.0001)

  0%|          | 0/5 [00:00<?, ?it/s]

<ipython-input-20-3b4ea88447e1>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(batch_text[0])
<ipython-input-17-35ede5d7ee8f>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x)
<ipython-input-17-35ede5d7ee8f>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float)


KeyboardInterrupt: ignored

In [ ]:
import pickle

In [ ]:
model = pickle.load(open("3nuc_model_batch16_hidden25", 'rb'))

In [ ]:
model

T_LSTM(
  (lstm): LSTM(1, 25, batch_first=True)
  (fc): Linear(in_features=25, out_features=2, bias=True)
)

Bakterie Gram ujemne

In [ ]:
get_accuracy(model, torchtext_train_dataloader)

<ipython-input-19-4eb31a9bb85a>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(batch_text[0])
<ipython-input-17-35ede5d7ee8f>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x)
<ipython-input-17-35ede5d7ee8f>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float)


0.7259030677073879

Bakterie Gram dodatnie

In [ ]:
get_accuracy(model, torchtext_train_dataloader2)

<ipython-input-19-4eb31a9bb85a>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(batch_text[0])
<ipython-input-17-35ede5d7ee8f>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x)
<ipython-input-17-35ede5d7ee8f>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float)


0.7249047013977128